<a href="https://colab.research.google.com/github/MartinTiong/CPE-311/blob/main/Hands_on_Activity_8_1_Aggregating_Pandas_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hands-on Activity 8.1: Aggregating Data with Pandas

1. With the earthquakes.csv file, select all the earthquakes in Japan with a magType of mb and a magnitude of 4.9 or greater.

2. Create bins for each full number of magnitude (for example, the first bin is 0-1, the second is 1-2, and so on) with a magType of ml and count how many are in each bin.

3. Using the faang.csv file, group by the ticker and resample to monthly frequency. Make the following aggregations:


- Mean of the opening price

- Maximum of the high price

- Minimum of the low price

- Mean of the closing price

- Sum of the volume traded

4. Build a crosstab with the earthquake data between the tsunami column and the magType column. Rather than showing the frequency count, show the maximum
magnitude that was observed for each combination. Put the magType along the columns.
5. Calculate the rolling 60-day aggregations of OHLC data by ticker for the FAANG data. Use the same aggregations as exercise no. 3.
6. Create a pivot table of the FAANG data that compares the stocks. Put the ticker in the rows and show the averages of the OHLC and volume traded data.
7.  Calculate the Z-scores for each numeric column of Netflix's data (ticker is NFLX) using apply().
8. Add event descriptions:
- Create a dataframe with the following three columns: ticker, date, and event. The columns should have the following values:

    *   ticker: 'FB'
    *   date: ['2018-07-25', '2018-03-19', '2018-03-20']
    event: ['Disappointing user growth announced after close.', 'Cambridge
- Set the index to ['date', 'ticker']
- Merge this data with the FAANG data using an outer join

9. Use the transform() method on the FAANG data to represent all the values in terms of the first date in the data. To do so, divide all the values for each ticker by the values
for the first date in the data for that ticker. This is referred to as an index, and the data for the first date is the base (https://ec.europa.eu/eurostat/statistics-explained/
index.php/ Beginners:Statisticalconcept-Indexandbaseyear). When data is in this format, we can easily see growth over time. Hint: transform() can take a function name.

In [9]:
import pandas as pd
import numpy as np


In [10]:
faang = pd.read_csv('/content/faang.csv')
earthquakes = pd.read_csv('/content/earthquakes.csv')

faang['date'] = pd.to_datetime(faang['date'])

In [14]:
filtered = earthquakes[
    (earthquakes["place"].str.contains("Japan", na=False)) &
    (earthquakes["magType"] == "mb") &
    (earthquakes["mag"] >= 4.9)
]
print(filtered)

      mag magType           time                         place  tsunami  \
1563  4.9      mb  1538977532250  293km ESE of Iwo Jima, Japan        0   
2576  5.4      mb  1538697528010    37km E of Tomakomai, Japan        0   
3072  4.9      mb  1538579732490     15km ENE of Hasaki, Japan        0   
3632  4.9      mb  1538450871260    53km ESE of Hitachi, Japan        0   

     parsed_place  
1563        Japan  
2576        Japan  
3072        Japan  
3632        Japan  


In [15]:
quakes = earthquakes[earthquakes["magType"] == "ml"].copy()


bins = np.arange(0, quakes["mag"].max() + 1, 1)
quakes["mag_bin"] = pd.cut(quakes["mag"], bins=bins)


quakeSS = quakes["mag_bin"].value_counts().sort_index()

In [13]:
faang["date"] = pd.to_datetime(faang["date"])
faang = faang.set_index("date")

monthly = (
    faang.groupby("ticker")
         .resample("ME")
         .agg({
             "open": "mean",
             "high": "max",
             "low": "min",
             "close": "mean",
             "volume": "sum"
         })
)

print(monthly)

                          open       high        low        close     volume
ticker date                                                                 
AAPL   2018-01-31   170.714690   176.6782   161.5708   170.699271  659679440
       2018-02-28   164.562753   177.9059   147.9865   164.921884  927894473
       2018-03-31   172.421381   180.7477   162.4660   171.878919  713727447
       2018-04-30   167.332895   176.2526   158.2207   167.286924  666360147
       2018-05-31   182.635582   187.9311   162.7911   183.207418  620976206
       2018-06-30   186.605843   192.0247   178.7056   186.508652  527624365
       2018-07-31   188.065786   193.7650   181.3655   188.179724  393843881
       2018-08-31   210.460287   227.1001   195.0999   211.477743  700318837
       2018-09-30   220.611742   227.8939   213.6351   220.356353  678972040
       2018-10-31   219.489426   231.6645   204.4963   219.137822  789748068
       2018-11-30   190.828681   220.6405   169.5328   190.246652  961321947

In [16]:
cross = pd.crosstab(
    earthquakes["tsunami"],
    earthquakes["magType"],
    values=earthquakes["mag"],
    aggfunc="max"
)

print(cross)

magType   mb  mb_lg    md   mh   ml  ms_20    mw  mwb  mwr  mww
tsunami                                                        
0        5.6    3.5  4.11  1.1  4.2    NaN  3.83  5.8  4.8  6.0
1        6.1    NaN   NaN  NaN  5.1    5.7  4.41  NaN  NaN  7.5


In [18]:
faang = faang.sort_index()

aggregations = (
    faang.groupby("ticker")
         .rolling("60D")
         .agg({
             "open": "mean", "high": "max", "low" : "min", "close": "mean","volume": "sum"
         })
)

print(aggregations)

                         open      high       low       close       volume
ticker date                                                               
AAPL   2018-01-02  166.927100  169.0264  166.0442  168.987200   25555934.0
       2018-01-03  168.089600  171.2337  166.0442  168.972500   55073833.0
       2018-01-04  168.480367  171.2337  166.0442  169.229200   77508430.0
       2018-01-05  168.896475  172.0381  166.0442  169.840675  101168448.0
       2018-01-08  169.324680  172.2736  166.0442  170.080040  121736214.0
...                       ...       ...       ...         ...          ...
NFLX   2018-12-24  283.509250  332.0499  233.6800  281.931750  525657894.0
       2018-12-26  281.844500  332.0499  231.2300  280.777750  520444588.0
       2018-12-27  281.070488  332.0499  231.2300  280.162805  532679805.0
       2018-12-28  279.916341  332.0499  231.2300  279.461341  521968250.0
       2018-12-31  278.430769  332.0499  231.2300  277.451410  476309676.0

[1255 rows x 5 columns]


In [19]:
pivot_table = pd.pivot_table(
    faang.reset_index(),
    index="ticker",
    values=["open", "high", "low", "close", "volume"],
    aggfunc="mean"
)

print(pivot_table)

              close         high          low         open        volume
ticker                                                                  
AAPL     186.986218   188.906858   185.135729   187.038674  3.402145e+07
AMZN    1641.726175  1662.839801  1619.840398  1644.072669  5.649563e+06
FB       171.510936   173.615298   169.303110   171.454424  2.768798e+07
GOOG    1113.225139  1125.777649  1101.001594  1113.554104  1.742645e+06
NFLX     319.290299   325.224583   313.187273   319.620533  1.147030e+07


In [20]:
nflx = faang[faang["ticker"] == "NFLX"]

z_scores = nflx.select_dtypes(include=np.number).apply(
    lambda x: (x - x.mean()) / x.std()
)

print(z_scores)

                open      high       low     close    volume
date                                                        
2018-01-02 -2.500753 -2.516023 -2.410226 -2.416644 -0.088760
2018-01-03 -2.380291 -2.423180 -2.285793 -2.335286 -0.507606
2018-01-04 -2.296272 -2.406077 -2.234616 -2.323429 -0.959287
2018-01-05 -2.275014 -2.345607 -2.202087 -2.234303 -0.782331
2018-01-08 -2.218934 -2.295113 -2.143759 -2.192192 -1.038531
...              ...       ...       ...       ...       ...
2018-12-24 -1.571478 -1.518366 -1.627197 -1.745946 -0.339003
2018-12-26 -1.735063 -1.439978 -1.677339 -1.341402  0.517040
2018-12-27 -1.407286 -1.417785 -1.495805 -1.302664  0.134868
2018-12-28 -1.248762 -1.289018 -1.297285 -1.292137 -0.085164
2018-12-31 -1.203817 -1.122354 -1.088531 -1.055420  0.359444

[251 rows x 5 columns]


In [21]:
events = pd.DataFrame({
    "ticker": ["FB", "FB", "FB"],
    "date": ["2018-07-25", "2018-03-19", "2018-03-20"],
    "event": [
        "Disappointing user growth announced after close.",
        "Cambridge Analytica story",
        "FTC investigation"
    ]
})

events["date"] = pd.to_datetime(events["date"])
events.set_index(["date", "ticker"], inplace=True)

faang_reset = faang.reset_index().set_index(["date", "ticker"])

merging = faang_reset.merge(events, how="outer", left_index=True, right_index=True)

print(merging)

                        open       high        low      close    volume event
date       ticker                                                            
2018-01-02 AAPL     166.9271   169.0264   166.0442   168.9872  25555934   NaN
           AMZN    1172.0000  1190.0000  1170.5100  1189.0100   2694494   NaN
           FB       177.6800   181.5800   177.5500   181.4200  18151903   NaN
           GOOG    1048.3400  1066.9400  1045.2300  1065.0000   1237564   NaN
           NFLX     196.1000   201.6500   195.4200   201.0700  10966889   NaN
...                      ...        ...        ...        ...       ...   ...
2018-12-31 AAPL     157.8529   158.6794   155.8117   157.0663  35003466   NaN
           AMZN    1510.8000  1520.7600  1487.0000  1501.9700   6954507   NaN
           FB       134.4500   134.6400   129.9500   131.0900  24625308   NaN
           GOOG    1050.9600  1052.7000  1023.5900  1035.6100   1493722   NaN
           NFLX     260.1600   270.1001   260.0000   267.6600  1

In [22]:
faang_reset = faang.reset_index()

q9 = (
    faang_reset.groupby("ticker")
               .transform(lambda x: x / x.iloc[0] if np.issubdtype(x.dtype, np.number) else x)
)

# Combine with ticker/date for readability
q9[["ticker", "date"]] = faang_reset[["ticker", "date"]]

print(q9)

           date      open      high       low     close    volume ticker
0    2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000     FB
1    2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000   AMZN
2    2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000   NFLX
3    2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000   AAPL
4    2018-01-02  1.000000  1.000000  1.000000  1.000000  1.000000   GOOG
...         ...       ...       ...       ...       ...       ...    ...
1250 2018-12-31  1.289078  1.277950  1.270386  1.263211  2.581007   AMZN
1251 2018-12-31  0.756697  0.741491  0.731907  0.722577  1.356624     FB
1252 2018-12-31  1.326670  1.339450  1.330468  1.331178  1.231791   NFLX
1253 2018-12-31  0.945640  0.938785  0.938375  0.929457  1.369681   AAPL
1254 2018-12-31  1.002499  0.986653  0.979296  0.972404  1.206986   GOOG

[1255 rows x 7 columns]
